In [29]:

# # IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# # TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# # THEN FEEL FREE TO DELETE THIS CELL.
# # NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# # ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# # NOTEBOOK.

# import os
# import sys
# from tempfile import NamedTemporaryFile
# from urllib.request import urlopen
# from urllib.parse import unquote, urlparse
# from urllib.error import HTTPError
# from zipfile import ZipFile
# import tarfile
# import shutil

# CHUNK_SIZE = 40960
# DATA_SOURCE_MAPPING = 'practical-ml-chocolate:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F50690%2F5357547%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240313%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240313T193310Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dca8ce8b41f41baa81b6a68c84c95f34d0651fb086aa8386cb39c596652994b7aaaf61f501b5adce8056a666a5a2fd37fa9e3ab06e2d364a6fb2f02d0a760702e02b0a093b45668ea4d7a4820e67302c4ae5dc60d93ef5b9bd05f321cdf4c801a07d1a541c4e4c59df95e7cd5ffaad319f1f1fb85b5710527a44e76078ee74203a9b8975eb75fede81cc689440dab4e33e3ff7d99de5dd036926fa5871a65e2f728104bb4d19796cdd53acc1d9bc13dee43ed93ce0d18a146220a44f9c64d281b4cae0e2c29459b405a55c25ef7c3ee833b7a1622de03432f023b08632903098ec6ff62d2bc3d9505894591436c9c9e3f43ab5272a75bf6444c03ed1afb08b569'

# KAGGLE_INPUT_PATH='/kaggle/input'
# KAGGLE_WORKING_PATH='/kaggle/working'
# KAGGLE_SYMLINK='kaggle'

# !umount /kaggle/input/ 2> /dev/null
# shutil.rmtree('/kaggle/input', ignore_errors=True)
# os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
# os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

# try:
#   os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
# except FileExistsError:
#   pass
# try:
#   os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
# except FileExistsError:
#   pass

# for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
#     directory, download_url_encoded = data_source_mapping.split(':')
#     download_url = unquote(download_url_encoded)
#     filename = urlparse(download_url).path
#     destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
#     try:
#         with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
#             total_length = fileres.headers['content-length']
#             print(f'Downloading {directory}, {total_length} bytes compressed')
#             dl = 0
#             data = fileres.read(CHUNK_SIZE)
#             while len(data) > 0:
#                 dl += len(data)
#                 tfile.write(data)
#                 done = int(50 * dl / int(total_length))
#                 sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
#                 sys.stdout.flush()
#                 data = fileres.read(CHUNK_SIZE)
#             if filename.endswith('.zip'):
#               with ZipFile(tfile) as zfile:
#                 zfile.extractall(destination_path)
#             else:
#               with tarfile.open(tfile.name) as tarfile:
#                 tarfile.extractall(destination_path)
#             print(f'\nDownloaded and uncompressed: {directory}')
#     except HTTPError as e:
#         print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
#         continue
#     except OSError as e:
#         print(f'Failed to load {download_url} to path {destination_path}')
#         continue

# print('Data source import complete.')


In [30]:
! pip install category_encoders catboost optuna pandas

In [31]:
import numpy as np
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
from category_encoders.cat_boost import CatBoostEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import TransformedTargetRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
import optuna as opt
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
from category_encoders.count import CountEncoder
from category_encoders.target_encoder import TargetEncoder
# pd.set_option('future.no_silent_downcasting', True)
opt.logging.set_verbosity(opt.logging.ERROR)

In [32]:
RANDOM_STATE=420
OPTIMIZE=False

In [33]:
TRAIN = "https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/chocolate_train.csv"
TEST = "https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/chocolate_test_new.csv"

In [34]:
df = pd.read_csv(TRAIN)

# 1. Анализ данных

* Company - компания производитель
* Specific Bean Origin - географический регион происхождения
* REF - параметр, определяющий момент внесения записи в базу данных (чем выше значение, тем "свежее" запись)
* Review - дата публикации отзыва
* Cocoa Percent - процентное содержание какао
* Company Location - страна производитель
* Bean Type - используемый сорт какао-бобов, если таковой имеется
* Broad Bean Origin - географический регион происхождения какао-бобов
* Rating - экспертный рейтинг

In [35]:
df.describe(include='all')

,Company,Specific Bean Origin,REF,Review,Cocoa Percent,Company Location,Rating,Bean Type,Broad Bean Origin
count,1255,1255,1255.000000,1255.000000,1255,1255,1255.000000,1254,1254
unique,376,762,NaN,NaN,40,58,NaN,38,84
top,Soma,Madagascar,NaN,NaN,70%,U.S.A.,NaN,,Venezuela
freq,35,39,NaN,NaN,461,521,NaN,628,144
mean,NaN,NaN,1045.152191,2012.382470,NaN,NaN,3.176494,NaN,NaN
std,NaN,NaN,551.284249,2.922499,NaN,NaN,0.478948,NaN,NaN
min,NaN,NaN,5.000000,2006.000000,NaN,NaN,1.000000,NaN,NaN
25%,NaN,NaN,593.000000,2010.000000,NaN,NaN,2.750000,NaN,NaN
50%,NaN,NaN,1077.000000,2013.000000,NaN,NaN,3.250000,NaN,NaN
75%,NaN,NaN,1514.000000,2015.000000,NaN,NaN,3.500000,NaN,NaN


* Есть пропуски в столбцах Bean Type, Broad Bean Origin. Для Bean Type если он не указан, то ставится пустая строка. Для Broad Bean Origin можно тоже применить этот способ.
* Процент кокао приведён в текстовом формате. Можно попробовать перевести в числовой.
* Категориальные признаки в основном имеют большое количество уникальных значений. Это стоит предусмотреть при кодировании признаков.

In [36]:
px.bar(df[['Company', 'Rating']].groupby('Company', as_index=False).mean(),
             x="Company",
             y="Rating",
             title="Средниее значение рейтинга в различных компаниях").show()
px.bar(df[['Company', 'Rating']].groupby('Company', as_index=False).min(),
             x="Company",
             y="Rating",
             title="Минимальное значение рейтинга в различных компаниях").show()
px.bar(df[['Company', 'Rating']].groupby('Company', as_index=False).max(),
             x="Company",
             y="Rating",
             title="Максимальное значение рейтинга в различных компаниях").show()
px.bar(x=df['Company'].unique(),
             y=[df[df['Company']==key].shape[0] for key in df['Company'].unique()],
             labels={
                     "y": "Количество",
                     "x": "Company",
                 },
            title="Количество значений компаний в тестовой выборке").show()

In [37]:
px.bar(df[['Specific Bean Origin', 'Rating']].groupby('Specific Bean Origin', as_index=False).mean(),
             x="Specific Bean Origin",
             y="Rating",
             title="Средниее значение рейтинга в различных регионах").show()
px.bar(df[['Specific Bean Origin', 'Rating']].groupby('Specific Bean Origin', as_index=False).min(),
             x="Specific Bean Origin",
             y="Rating",
             title="Минимальное значение рейтинга в различных регионах").show()
px.bar(df[['Specific Bean Origin', 'Rating']].groupby('Specific Bean Origin', as_index=False).max(),
             x="Specific Bean Origin",
             y="Rating",
             title="Максимальное значение рейтинга в различных регионах").show()
px.bar(x=df['Specific Bean Origin'].unique(),
             y=[df[df['Specific Bean Origin']==key].shape[0] for key in df['Specific Bean Origin'].unique()],
             labels={
                     "y": "Количество",
                     "x": "Specific Bean Origin",
                 },
            title="Количество значений регионов в тестовой выборке").show()

In [38]:
px.scatter(df, y='REF', x="Rating",
             title="Зависимость рейтинга от момента записи в базу данных").show()

После 252 перестали выставлять 1 в рейтинге. Похожее поведение с 1.5 и 5 в рейтинге.

При обучении модели можем переобучится и при появлении новых данных не выставить корректный рейтинг.

In [39]:
px.scatter(df, y='Review', x="Rating",
             title="Зависимость рейтинга от даты публикации отзыва").show()

Поведение даты публикации отзыва похоже на момент записи в базу, что логично

In [40]:
px.bar(df[['Cocoa Percent', 'Rating']].groupby('Cocoa Percent', as_index=False).mean(),
             x="Cocoa Percent",
             y="Rating",
             title="Средниее значение рейтинга для разного процента какао").show()
px.bar(df[['Cocoa Percent', 'Rating']].groupby('Cocoa Percent', as_index=False).min(),
             x="Cocoa Percent",
             y="Rating",
             title="Минимальное значение рейтинга для разного процента какао").show()
px.bar(df[['Cocoa Percent', 'Rating']].groupby('Cocoa Percent', as_index=False).max(),
             x="Cocoa Percent",
             y="Rating",
             title="Максимальное значение рейтинга для разного процента какао").show()
px.bar(x=df['Cocoa Percent'].unique(),
             y=[df[df['Cocoa Percent']==key].shape[0] for key in df['Cocoa Percent'].unique()],
             labels={
                     "y": "Количество",
                     "x": "Cocoa Percent",
                 },
            title="Количество значений разного процента какао в тестовой выборке").show()

In [41]:
px.bar(df[['Bean Type', 'Rating']].groupby('Bean Type', as_index=False).mean(),
             x="Bean Type",
             y="Rating",
             title="Средниее значение рейтинга для разного сорта бобов").show()
px.bar(df[['Bean Type', 'Rating']].groupby('Bean Type', as_index=False).min(),
             x="Bean Type",
             y="Rating",
             title="Минимальное значение рейтинга для разного сорта бобов").show()
px.bar(df[['Bean Type', 'Rating']].groupby('Bean Type', as_index=False).max(),
             x="Bean Type",
             y="Rating",
             title="Максимальное значение рейтинга для разного сорта бобов").show()
px.bar(x=df['Bean Type'].unique(),
             y=[df[df['Bean Type']==key].shape[0] for key in df['Bean Type'].unique()],
             labels={
                     "y": "Количество",
                     "x": "Bean Type",
                 },
            title="Количество сортов бобов в тестовой выборке").show()

Основное количество объектов в тестовой выборке без указания сорта бобов

In [42]:
px.bar(df[['Broad Bean Origin', 'Rating']].groupby('Broad Bean Origin', as_index=False).mean(),
             x="Broad Bean Origin",
             y="Rating",
             title="Средниее значение рейтинга для разного региона происхождения бобов").show()
px.bar(df[['Broad Bean Origin', 'Rating']].groupby('Broad Bean Origin', as_index=False).min(),
             x="Broad Bean Origin",
             y="Rating",
             title="Минимальное значение рейтинга для разного региона происхождения бобов").show()
px.bar(df[['Broad Bean Origin', 'Rating']].groupby('Broad Bean Origin', as_index=False).max(),
             x="Broad Bean Origin",
             y="Rating",
             title="Максимальное значение рейтинга для разного региона происхождения бобов").show()
px.bar(x=df['Broad Bean Origin'].unique(),
             y=[df[df['Broad Bean Origin']==key].shape[0] for key in df['Broad Bean Origin'].unique()],
             labels={
                     "y": "Количество",
                     "x": "Bean Type",
                 },
            title="Количество регионов происхождения бобов в тестовой выборке").show()

Многие регионы представлены в выборке по одному, так как мало данных можно заменить их пустой строкой

In [43]:
temp = df['Rating'].unique()
temp.sort()
px.line( x=temp,
        y=[df[df['Rating'] == key].shape[0] for key in temp],
        labels={
            "y": "Количество",
            "x": "Rating",
        },
        title='Распределение значений целевой переменной')

# 2. Подготовка данных

* Заполнение недастающих значений в столбцах Bean Type, Broad Bean Origin пустыми строками и поиск похожих пар значений с целью подмены пустых строк
* Перевод процентов содержания кокао в числовой формат.
* Удаление столбцов REF и Review.
* Разделение данных на train и test.

In [44]:
def prepareData(df):
    df['Bean Type'] = df['Bean Type'].fillna(value='\xa0')
    df['Broad Bean Origin'] = df['Broad Bean Origin'].fillna(value='\xa0')
    df['Cocoa Percent'] = df['Cocoa Percent'].map(lambda x: float(x.split('%')[0]))

    for [i, row] in df[df['Broad Bean Origin'] == '\xa0'].iterrows():
        rating = df[(df['Bean Type'] == row['Bean Type']) & (df['Company'] == row['Company'])]['Broad Bean Origin'].where(lambda x : x != '\xa0').value_counts().index
        if not rating.shape[0]:
            rating = df[(df['Bean Type'] == row['Bean Type']) & (df['Cocoa Percent'] == row['Cocoa Percent'])]['Broad Bean Origin'].where(lambda x : x != '\xa0').value_counts().index
        if not rating.shape[0]:
            rating = df[df['Bean Type'] == row['Bean Type']]['Broad Bean Origin'].where(lambda x : x != '\xa0').value_counts().index
        if rating.shape[0]:
            df.loc[i, 'Broad Bean Origin'] = rating[0]

    for [i, row] in df[df['Bean Type'] == '\xa0'].iterrows():
        rating = df[(df['Broad Bean Origin'] == row['Broad Bean Origin']) & (df['Company'] == row['Company'])]['Bean Type'].where(lambda x : x != '\xa0').value_counts().index
        if not rating.shape[0]:
            rating = df[(df['Broad Bean Origin'] == row['Broad Bean Origin']) & (df['Cocoa Percent'] == row['Cocoa Percent'])]['Bean Type'].where(lambda x : x != '\xa0').value_counts().index
        if not rating.shape[0]:
            rating = df[df['Broad Bean Origin'] == row['Broad Bean Origin']]['Bean Type'].where(lambda x : x != '\xa0').value_counts().index
        if rating.shape[0]:
            df.loc[i, 'Bean Type'] = rating[0]
    return df

In [45]:
df = prepareData(df)

In [46]:
px.bar(df[['Bean Type', 'Rating']].groupby('Bean Type', as_index=False).mean(),
             x="Bean Type",
             y="Rating",
             title="Средниее значение рейтинга для разного сорта бобов").show()
px.bar(df[['Bean Type', 'Rating']].groupby('Bean Type', as_index=False).min(),
             x="Bean Type",
             y="Rating",
             title="Минимальное значение рейтинга для разного сорта бобов").show()
px.bar(df[['Bean Type', 'Rating']].groupby('Bean Type', as_index=False).max(),
             x="Bean Type",
             y="Rating",
             title="Максимальное значение рейтинга для разного сорта бобов").show()
px.bar(x=df['Bean Type'].unique(),
             y=[df[df['Bean Type']==key].shape[0] for key in df['Bean Type'].unique()],
             labels={
                     "y": "Количество",
                     "x": "Bean Type",
                 },
            title="Количество сортов бобов в тестовой выборке").show()

In [47]:
X = df.drop(['REF', 'Review', 'Rating'], axis=1)
y = df['Rating']

In [48]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

temp = ytrain.unique()
temp.sort()
px.line(
    x=temp,
    y=[ytrain[ytrain == key].shape[0] for key in temp],
    labels={
        'y': 'Количество',
        'x': 'Rating'
    },
    title='Распределение целевой переменной в обучающей выборке')

# 3. Выбор модели и подбор гиперпараметров

In [49]:
def transform_target(target):
    return target

def inverse_target(target):
    return np.round(target / 0.25) * 0.25

In [50]:
class FeatureCoding:
    def __init__(self,
        cat_cols=[
            "Company",
            "Specific Bean Origin",
            "Company Location",
            "Bean Type",
            "Broad Bean Origin"
        ],
        filter_cols=[
            "Company",
            "Specific Bean Origin",
            "Company Location",
            "Bean Type",
            "Broad Bean Origin"
        ],
        threshold=1,
        min_samples_leaf=20, smoothing=10,
        ):
        self.cat_cols = cat_cols
        self.threshold = threshold
        self.skip_values= {key: [] for key in filter_cols}
        self.count_encoder = CountEncoder(cols=cat_cols)
        self.min_max_scaler = MinMaxScaler()
        self.encoder = TargetEncoder(cols=cat_cols, min_samples_leaf=20, smoothing=10,)
    def fit(self, x, y):
        temp = x.copy()
        self.count_encoder.fit(temp[self.cat_cols])
        count = self.count_encoder.transform(temp[self.cat_cols])
        self.min_max_scaler.fit(count.values)
        count = self.min_max_scaler.transform(count.values)
        temp[[f'Count - {column}' for column in self.cat_cols]] = count
        if bool(self.skip_values):
            self._collect_skip_values(temp)
        self.encoder.fit(temp, y)
    def transform(self, x):
        temp = x.copy()
        count = self.count_encoder.transform(temp[self.cat_cols])
        count = self.min_max_scaler.transform(count.values)
        temp[[f'Count - {column}' for column in self.cat_cols]] = count
        if bool(self.skip_values):
            temp = self._skip_values(temp)
        return self.encoder.transform(temp)
    def fit_transform(self, x, y):
        self.fit(x, y)
        return self.transform(x)
    def _collect_skip_values(self, x):
        temp = x.copy()
        for column in self.skip_values.keys():
            values = temp[column].unique()
            for value in values:
                if temp[temp[column] == value].shape[0] <= self.threshold:
                    self.skip_values[column].append(value)
        return temp
    def _skip_values(self, x):
        temp = x.copy()
        for [column, values] in self.skip_values.items():
            temp[column] = temp[column].map(lambda x: '\xa0' if x in values else x)
        return temp

In [51]:
class GenerateFeatures:
    def __init__(self,
                 random_state=470,
                 n_clusters=40,
                 n_init=1,
                 degree=2,
                 interaction_only=False,
                 skip_features=[]
                ):
        self.kmeans = KMeans(
            n_clusters=n_clusters,
            random_state=random_state,
            n_init=n_init)
        self.poly = PolynomialFeatures(degree, interaction_only=interaction_only, include_bias=False)
        self.poly.set_output(transform='pandas')
        self.encoder = FeatureCoding(["Clusters"], [])
        self.skip_features=skip_features
    def fit(self, x, y):
        temp = x[x.columns]
        self.kmeans.fit(temp)
        temp["Clusters"] = self.kmeans.predict(temp)
        self.encoder.fit(temp, y)
        temp = self.encoder.transform(temp)
        self.poly.fit(temp)
    def transform(self, x):
        temp = x[x.columns]
        temp['Clusters'] = self.kmeans.predict(temp)
        temp = self.encoder.transform(temp)
        temp = self.poly.transform(temp)
        return temp.drop(self.skip_features, axis=1)
    def fit_transform(self, x, y):
        self.fit(x, y)
        return self.transform(x)

In [52]:
def objective_catboost (trial):
    params = {
        "iterations": trial.suggest_int("iterations", 1500, 2500, step=100),
        "learning_rate": trial.suggest_float("learning_rate", 0.0001, 0.01, step=0.0001),
        "depth": trial.suggest_int("depth", 1, 5),
        "subsample": trial.suggest_float("subsample", 0.1, 1.0, step=0.01),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.5, step=0.01),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 10),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
        "random_state": trial.suggest_int("random_state_model", 1, 1000),
        "silent":True,
        "loss_function": "RMSE",
    }
    if params['grow_policy'] == "SymmetricTree":
        params["boosting_type"]= trial.suggest_categorical("boosting_type", ["Ordered", "Plain"])
    else:
        params["boosting_type"] = "Plain"
    pipe = Pipeline([
        ('feature_coding', FeatureCoding(
            threshold=trial.suggest_int("threshold", 0, 5),
            min_samples_leaf=trial.suggest_int("min_samples_leaf", 1, 15),
            smoothing=trial.suggest_float("smoothing", 0.1, 10, step=0.1),
        )),
        ('generate_features', GenerateFeatures(
            random_state=trial.suggest_int("random_state_generator", 1, 1000),
            n_clusters=trial.suggest_int("n_clusters", 1, 500),
            n_init=trial.suggest_int("n_init", 1, 10),
            degree=trial.suggest_int("degree", 1, 3),
        )),
        ('model', CatBoostRegressor(**params))])
    model = TransformedTargetRegressor(
        pipe,
        func=transform_target,
        inverse_func=inverse_target
    )
    return cross_val_score(model, Xtrain, ytrain, cv=3, scoring='r2').mean()

if OPTIMIZE:
    study = opt.create_study(direction="maximize")
    study.optimize(objective_catboost, timeout=60*60)
    print(study.best_params)

In [53]:
params = {
    'iterations': 1830,
    'learning_rate': 0.0043,
    'depth': 2,
    'subsample': 0.33,
    'colsample_bylevel': 0.08,
    'min_data_in_leaf': 9,
    'grow_policy': 'SymmetricTree',
    'boosting_type': 'Plain',
    'random_state': 171,
    'loss_function': "RMSE",
    'silent': True
}
pipe = Pipeline([
    ('feature_coding', FeatureCoding(threshold=1, min_samples_leaf=14, smoothing=2.5)),
    ('generate_features', GenerateFeatures(
        random_state=300,
        n_clusters=253,
        n_init=10,
        degree=1,
        skip_features=['Count - Specific Bean Origin', 'Count - Company Location', 'Count - Bean Type', 'Count - Broad Bean Origin', 'Count - Clusters']
    )),
    ('model', CatBoostRegressor(**params))
])
cat = TransformedTargetRegressor(
    pipe,
    func=transform_target,
    inverse_func=inverse_target
)
cat.fit(Xtrain, ytrain)
pred = cat.predict(Xtest)
r2_score(ytest, pred)

c:\Users\kuche\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.



0.144781076024168

In [54]:
columns = cat.regressor_['generate_features'].transform(
            cat.regressor_['feature_coding'].transform(Xtest)
).columns

temp = pd.DataFrame(
    [
        *[[i, value, 'value'] for [i, value] in enumerate(ytest)],
        *[[i, value, 'prediction'] for [i, value] in enumerate(pred)]
    ],
    columns=['id', 'value', 'type'])
px.line(temp, x='id', y='value', color='type', title='Ошибка прогнозов').show()
temp = pd.DataFrame(
    [columns for columns in zip(columns, cat.regressor_['model'].feature_importances_)],
    columns=['feature', 'value']
)
px.bar(temp, y='value', x='feature', title='Важность признаков для модели').show()
print('Признаки с маленьким весом:\n', [columns[i] for [i, v] in enumerate(cat.regressor_['model'].feature_importances_) if v < 1])

Признаки с маленьким весом:
 []


In [55]:
def objective_xgboost (trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.04, step=0.001),
        'n_estimators': trial.suggest_int('n_estimators', 10, 1500, step=10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.01, 1.0, step=0.01),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0, step=0.01),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0, step=0.01),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0, step=0.01),
        'random_state': RANDOM_STATE,
        "verbosity": 0
    }
    pipe = Pipeline([
        ('feature_coding', FeatureCoding(
            threshold=trial.suggest_int("threshold", 0, 5),
            min_samples_leaf=trial.suggest_int("min_samples_leaf", 1, 15),
            smoothing=trial.suggest_float("smoothing", 0.1, 10, step=0.1),
        )),
        ('generate_features', GenerateFeatures(
            random_state=trial.suggest_int("random_state_generator", 1, 1000),
            n_clusters=trial.suggest_int("n_clusters", 1, 100),
            n_init=trial.suggest_int("n_init", 1, 10),
            degree=trial.suggest_int("degree", 1, 3),
        )),
        ('model', XGBRegressor(**params))])
    model = TransformedTargetRegressor(
        pipe,
        func=transform_target,
        inverse_func=inverse_target
    )
    return cross_val_score(model, Xtrain, ytrain, cv=3, scoring='r2').mean()

if OPTIMIZE:
    study = opt.create_study(direction="maximize")
    study.optimize(objective_xgboost, timeout=60*60)
    print(study.best_params)

In [56]:
params = {
    'max_depth': 9,
    'learning_rate': 0.011,
    'n_estimators': 470,
    'min_child_weight':9,
    'gamma': 0.68,
    'subsample': 0.85,
    'colsample_bytree': 0.093,
    'reg_alpha': 0.17,
    'reg_lambda': 0.73,
}
pipe = Pipeline([
    ('feature_coding', FeatureCoding(threshold=1, min_samples_leaf=14, smoothing=2.5)),
    ('generate_features', GenerateFeatures(
        random_state=RANDOM_STATE,
        n_clusters=253,
        n_init=10,
        degree=1,

        skip_features=[]
    )),
    ('model', XGBRegressor(**params))])
xgb = TransformedTargetRegressor(
    pipe,
    func=transform_target,
    inverse_func=inverse_target
)
xgb.fit(Xtrain, ytrain)
pred = xgb.predict(Xtest)
r2_score(ytest, pred)

c:\Users\kuche\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.



AttributeError: 'super' object has no attribute '__sklearn_tags__'

In [57]:
columns = xgb.regressor_['generate_features'].transform(
            xgb.regressor_['feature_coding'].transform(Xtest)
).columns
temp = pd.DataFrame(
    [
        *[[i, value, 'value'] for [i, value] in enumerate(ytest)],
        *[[i, value, 'prediction'] for [i, value] in enumerate(pred)]
    ],
    columns=['id', 'value', 'type'])
px.line(temp, x='id', y='value', color='type', title='Ошибка прогнозов').show()
temp = pd.DataFrame(
    [columns for columns in zip(columns, xgb.regressor_['model'].feature_importances_)],
    columns=['feature', 'value']
)
px.bar(temp, y='value', x='feature', title='Важность признаков для модели').show()

In [58]:
pred = np.round((0.6 * cat.predict(Xtest) + 0.4 * xgb.predict(Xtest)) / 0.25) * 0.25
r2_score(ytest, pred)

AttributeError: 'super' object has no attribute '__sklearn_tags__'

# 4. Применение модели

In [ ]:
test_df = pd.read_csv('/kaggle/input/practical-ml-chocolate/chocolate_test_new.csv')

test_df = prepareData(test_df)

X_test = test_df.drop(['REF', 'Review'], axis=1)

In [ ]:
cat.fit(X, y)
pred = cat.predict(X_test)
submission = pd.DataFrame([[i, v] for [i, v] in enumerate(pred)], columns=['id', 'Rating'])
submission.to_csv("submission.csv", index=False)